# Import modules

In [12]:
import numpy as np
import pandas as pd
import datetime as datetime
from sklearn.ensemble import RandomForestClassifier

## Convert time: timestamp and registration date to unix timestamp

In [13]:
# Read the split dataset
# Available on github/data/splits
df_train = pd.read_csv('bpi2017_train.csv')
df_test = pd.read_csv('bpi2017_test.csv')
df_val = pd.read_csv('bpi2017_val.csv')

# Perform conversion
df_train['Date'] = np.array(df_train['time:timestamp'].values, dtype = 'datetime64').astype(datetime.datetime)

df_train['time:unix'] = (df_train['Date'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

df_test['Date'] = np.array(df_test['time:timestamp'].values, dtype = 'datetime64').astype(datetime.datetime)

df_test['time:unix'] = (df_test['Date'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

df_val['Date'] = np.array(df_val['time:timestamp'].values, dtype = 'datetime64').astype(datetime.datetime)

df_val['time:unix'] = (df_val['Date'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

/var/folders/lh/s2l49w850tq2lvbfyqst82j00000gn/T/ipykernel_26889/2956044118.py:8: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  df_train['Date'] = np.array(df_train['time:timestamp'].values, dtype = 'datetime64').astype(datetime.datetime)
/var/folders/lh/s2l49w850tq2lvbfyqst82j00000gn/T/ipykernel_26889/2956044118.py:12: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  df_test['Date'] = np.array(df_test['time:timestamp'].values, dtype = 'datetime64').astype(datetime.datetime)
/var/folders/lh/s2l49w850tq2lvbfyqst82j00000gn/T/ipykernel_26889/2956044118.py:16: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  df_val['Date'] = np.array(df_val['time:timestamp'].values, dtype = 'datetime64').astype(datetime.datetime)


In [14]:
# Define mapping 
mapping_train = {item:i for i, item in enumerate(df_train["lifecycle:transition"].unique())}
mapping_test = {item:i for i, item in enumerate(df_test["lifecycle:transition"].unique())}
mapping_val = {item:i for i, item in enumerate(df_val["lifecycle:transition"].unique())}

# Apply mapping
df_train["transition"] = df_train["lifecycle:transition"].apply(lambda x: mapping_train[x])
df_test["transition"] = df_test["lifecycle:transition"].apply(lambda x: mapping_test[x])
df_val["transition"] = df_val["lifecycle:transition"].apply(lambda x: mapping_val[x])

In [15]:
# Drop all NaNs
# df_train.dropna(axis=0, inplace=True)
# df_test.dropna(axis=0, inplace=True)
# df_val.dropna(axis=0, inplace=True)

# Define predictors
predictors=['time:unix','transition']

# Define the classifier
rfc = RandomForestClassifier(n_estimators=50)

# Fit the model
rfc.fit(df_train[predictors], df_train['concept:name'])

RandomForestClassifier(n_estimators=50)

In [16]:
pred_test = rfc.predict(df_test[predictors])
pred_val = rfc.predict(df_val[predictors])

df_test['predicted_action'] = pred_test
df_val['predicted_action'] = pred_val

In [17]:
from sklearn import metrics
actions_taken = df_val['concept:name']
actions_taken = actions_taken[1:]

actions_pred = df_val['predicted_action']
actions_pred = actions_pred[:-1]

test = pd.concat([actions_taken, actions_pred], axis=1)
test.dropna(axis=0, inplace=True)

print("Accuracy: ", metrics.accuracy_score(test['concept:name'], test['predicted_action']))

Accuracy:  0.2177497919341415
